# Проект e-commerce.

In [11]:
# подгружаем необходимые библиотеки
import pandas as pd 
from operator import itemgetter
from datetime import timedelta
import requests
from urllib.parse import urlencode 


Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз? 

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

3. По каждому товару определить, в какой день недели товар чаще всего покупается.

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут.

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.

Файлы:

olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей
customer_id — позаказный идентификатор пользователя

customer_unique_id —  уникальный идентификатор пользователя  (аналог номера паспорта)

customer_zip_code_prefix —  почтовый индекс пользователя

customer_city —  город доставки пользователя

customer_state —  штат доставки пользователя

olist_orders_dataset.csv —  таблица заказов
order_id —  уникальный идентификатор заказа (номер чека)

customer_id —  позаказный идентификатор пользователя

order_status —  статус заказа

order_purchase_timestamp —  время создания заказа

order_approved_at —  время подтверждения оплаты заказа

order_delivered_carrier_date —  время передачи заказа в логистическую службу

order_delivered_customer_date —  время доставки заказа

order_estimated_delivery_date —  обещанная дата доставки

olist_order_items_dataset.csv —  товарные позиции, входящие в заказы
order_id —  уникальный идентификатор заказа (номер чека)

order_item_id —  идентификатор товара внутри одного заказа

product_id —  ид товара (аналог штрихкода)

seller_id — ид производителя товара

shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике

price —  цена за единицу товара

freight_value —  вес товара

— Пример структуры данных можно визуализировать по order_id == 00143d0f86d6fbd9f9b38ab440ac16f5

Уникальные статусы заказов в таблице olist_orders_dataset:

created —  создан
approved —  подтверждён
invoiced —  выставлен счёт
processing —  в процессе сборки заказа
shipped —  отгружен со склада
delivered —  доставлен пользователю
unavailable —  недоступен
canceled —  отменён

In [5]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_olist_customers = 'https://disk.yandex.ru/d/QXrVDUoQvFhIcg'  # ссылка на файл  olist_customers_datase.csv
public_key_olist_orders = 'https://disk.yandex.ru/d/0zWmh1bYP6REbw'  # ссылка на файл olist_orders_dataset.csv
public_key_olist_order_items = 'https://disk.yandex.ru/d/xCsQ0FiC-Ue4Lg'  # ссылка на файл olist_order_items_dataset.csv

# Получаем загрузочные ссылки
final_url_olist_customers = base_url + urlencode(dict(public_key=public_key_olist_customers))
response_olist_customers = requests.get(final_url_olist_customers)
download_url_olist_customers = response_olist_customers.json()['href']

final_url_olist_orders = base_url + urlencode(dict(public_key=public_key_olist_orders))
response_olist_orders = requests.get(final_url_olist_orders)
download_url_olist_orders = response_olist_orders.json()['href']

final_url_olist_order_items = base_url + urlencode(dict(public_key=public_key_olist_order_items))
response_olist_order_items = requests.get(final_url_olist_order_items)
download_url_olist_order_items = response_olist_order_items.json()['href']

In [8]:
customers_dataset = pd.read_csv(download_url_olist_customers, encoding='windows-1251')
# подгружаем данные о покупателях

In [57]:
orders_dataset = pd.read_csv(download_url_olist_orders, encoding='windows-1251') 
# подгружаем данные о заказах

In [58]:
dataset_with_customers_and_orders = pd.merge(customers_dataset, orders_dataset, on='customer_id', how='inner') 
# соединяем данные о покупателях и заказах по customer_id ( id покупателя)
dataset_with_customers_and_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


-------------------------------------------------------------------------------------------------------------------------------

 1. Сколько у нас пользователей, которые совершили покупку только один раз?

In [59]:
customers_1_order = dataset_with_customers_and_orders.query('order_status != "canceled" & order_status != "unavailable"') \
    .groupby('customer_unique_id').agg({'order_id': 'count'}).reset_index()
# исключили недоступные и отмененные заказы из таблицы и посчитали количество заказов каждого пользователя
customers_1_order = customers_1_order.query('order_id == 1') # согласно условию, фильтруем пользователей по "1" покупке
customers_1_order.shape[0] # количество пользователей, совершивших только одну покупку "92102"

92102

-------------------------------------------------------------------------------------------------------------------------------

 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [60]:
canceled_and_unavailable_orders = dataset_with_customers_and_orders.query('order_status == "unavailable" or order_status == "canceled"') \
    .sort_values('order_estimated_delivery_date')
canceled_and_unavailable_orders = canceled_and_unavailable_orders[['order_status', 'order_estimated_delivery_date']]
canceled_and_unavailable_orders['order_estimated_delivery_date'] = pd.to_datetime(canceled_and_unavailable_orders['order_estimated_delivery_date'])
canceled_and_unavailable_orders = canceled_and_unavailable_orders.groupby(pd.Grouper(freq='M', key='order_estimated_delivery_date')) \
    .order_status.value_counts().to_frame().rename(columns={'order_status': 'quantity'}).reset_index() 
# таблица отображает количество отмененных и недоступных заказов за каждый месяц
canceled_and_unavailable_orders.head()

,order_estimated_delivery_date,order_status,quantity
0,2016-09-30,canceled,1
1,2016-10-31,canceled,3
2,2016-11-30,canceled,14
3,2016-11-30,unavailable,5
4,2016-12-31,canceled,8


In [61]:
mean_order = canceled_and_unavailable_orders.groupby('order_status') \
    .agg({'quantity': 'mean'}) \
    .rename(columns={'quantity': 'mean_value'}) \
    .reset_index()
mean_order.round(0) 
# берем из предыдущей таблицы общую информацию по заказам за все время, группируем по статусу и рассчитаем среднее значение 
# за данное время, округлив до целого числа

,order_status,mean_value
0,canceled,24.0
1,unavailable,29.0


-------------------------------------------------------------------------------------------------------------------------------

 3. По каждому товару определить, в какой день недели товар чаще всего покупается.

In [10]:
olist_order_items_dataset = pd.read_csv(download_url_olist_order_items)
# подгружаем данные о товарных позициях

In [63]:
dataset_with_customers_and_orders_items = pd.merge(dataset_with_customers_and_orders, olist_order_items_dataset, on='order_id',how='left')
# присоединяем к данным о покупателях и заказах данные о товарных позициях, входящих в заказ
dataset_with_customers_and_orders_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [64]:
dataset_with_customers_and_orders_items['order_purchase_timestamp'] = pd.to_datetime(dataset_with_customers_and_orders_items['order_purchase_timestamp'])
dataset_with_customers_and_orders_items['weekday'] = dataset_with_customers_and_orders_items['order_purchase_timestamp'].dt.day_name()
# добавляем колонку с днями недели
dataset_with_customers_and_orders_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,weekday
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,Saturday
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,Tuesday
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,Sunday


In [65]:
weekday_products = dataset_with_customers_and_orders_items.groupby(['product_id', 'weekday']).agg({'seller_id': 'count'}) \
    .rename(columns={'seller_id': 'quantity'}).reset_index()
weekday_products # общая таблица по датам, продукту, количеству приобретенных продуктов

,product_id,weekday,quantity
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1
2,0009406fd7479715e4bef61dd91f2462,Thursday,1
3,000b8f95fcb9e0096488278317764d19,Friday,1
4,000b8f95fcb9e0096488278317764d19,Wednesday,1
...,...,...,...
61225,fffdb2d0ec8d6a61f0a0a0db3f25b441,Friday,1
61226,fffdb2d0ec8d6a61f0a0a0db3f25b441,Sunday,1
61227,fffdb2d0ec8d6a61f0a0a0db3f25b441,Thursday,1
61228,fffdb2d0ec8d6a61f0a0a0db3f25b441,Tuesday,2


In [67]:
without_duplicates_products = weekday_products.drop_duplicates(subset='product_id') 
# убираем дубликаты для дальнейшего мерджа с таблицей, где будет самый часто покупаемый продукт
without_duplicates_products = without_duplicates_products[['product_id', 'weekday']] # оставляем необходимые данные
without_duplicates_products

,product_id,weekday
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday
1,00088930e925c41fd95ebfe695fd2655,Tuesday
2,0009406fd7479715e4bef61dd91f2462,Thursday
3,000b8f95fcb9e0096488278317764d19,Friday
5,000d9be29b5207b54e86aa1b1ac54872,Tuesday
...,...,...
61221,fff6177642830a9a94a0f2cba5e476d1,Saturday
61223,fff81cc3158d2725c0655ab9ba0f712c,Monday
61224,fff9553ac224cec9d15d49f5a263411f,Friday
61225,fffdb2d0ec8d6a61f0a0a0db3f25b441,Friday


In [68]:
max_product = weekday_products.groupby('product_id', as_index=False).agg({'quantity': 'max'}) 
# агрегируем данные и получаем самый часто покупаемый продукт
max_product

,product_id,quantity
0,00066f42aeeb9f3007548bb9d3f33c38,1
1,00088930e925c41fd95ebfe695fd2655,1
2,0009406fd7479715e4bef61dd91f2462,1
3,000b8f95fcb9e0096488278317764d19,1
4,000d9be29b5207b54e86aa1b1ac54872,1
...,...,...
32946,fff6177642830a9a94a0f2cba5e476d1,1
32947,fff81cc3158d2725c0655ab9ba0f712c,1
32948,fff9553ac224cec9d15d49f5a263411f,1
32949,fffdb2d0ec8d6a61f0a0a0db3f25b441,2


In [69]:
df_max_product_result = without_duplicates_products.merge(max_product, how='inner', on='product_id') 
df_max_product_result # получаем результирующую таблицу с самыми часто покупаемыми продуктами в каждый из дней

,product_id,weekday,quantity
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1
2,0009406fd7479715e4bef61dd91f2462,Thursday,1
3,000b8f95fcb9e0096488278317764d19,Friday,1
4,000d9be29b5207b54e86aa1b1ac54872,Tuesday,1
...,...,...,...
32946,fff6177642830a9a94a0f2cba5e476d1,Saturday,1
32947,fff81cc3158d2725c0655ab9ba0f712c,Monday,1
32948,fff9553ac224cec9d15d49f5a263411f,Friday,1
32949,fffdb2d0ec8d6a61f0a0a0db3f25b441,Friday,2


-------------------------------------------------------------------------------------------------------------------------------

 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?

In [70]:
dataset_with_customers_and_orders_items.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,weekday
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday


In [71]:
dataset_with_customers_and_orders_items['order_delivered_customer_date'] = pd.to_datetime(dataset_with_customers_and_orders_items['order_delivered_customer_date'])
# переводим данные по доставке заказа в формат datetime
dataset_with_customers_and_orders_items['order_delivered_month'] = dataset_with_customers_and_orders_items['order_delivered_customer_date'].dt.month_name() # фиксируем название месяца в таблице
dataset_with_customers_and_orders_items['days_in_month'] = dataset_with_customers_and_orders_items['order_delivered_customer_date'].dt.days_in_month # вычисляем сколько дней в каждом месяце
dataset_with_customers_and_orders_items['week_in_month'] = dataset_with_customers_and_orders_items.days_in_month / 7 
# добавляем количество недель в месяце, согласно заданию
df_4 = dataset_with_customers_and_orders_items[['customer_unique_id', 'order_id', 'order_delivered_month', 'week_in_month']] 
# отбираем необходимые данные
df_4 = df_4.groupby(['customer_unique_id', 'order_delivered_month', 'week_in_month'], as_index=False).agg({'order_id': 'count'}) \
.rename(columns={'order_id': 'orders_quantity'})
# вычисляем при помощи агрегации количество приобретенных заказов в месяц
df_4['mean_week'] = df_4.orders_quantity / df_4.week_in_month 
# находим среднее значение покупки в неделю, исходя из кол-ва заказов и среднего кол-ва недель в месяц
df_4_mean_res = df_4[['customer_unique_id', 'order_delivered_month', 'mean_week']] 
# фильтруем данные согласно условию, оставляем id клиента, месяц покупки и среднее значение покупки товара в неделю от месяца
df_4_mean_res.head()


,customer_unique_id,order_delivered_month,mean_week
0,0000366f3b9a7992bf8c76cfdf3221e2,May,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,May,0.225806
2,0000f46a3911fa3c0805444483337064,April,0.233333
3,0000f6ccb0745a6a4b88665a16c9f078,November,0.233333
4,0004aac84e0df4da2b147fca70cf8255,November,0.233333


-------------------------------------------------------------------------------------------------------------------------------

 5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

In [72]:
df_5 = dataset_with_customers_and_orders_items.copy(deep=True).query('order_delivered_customer_date != "NaN"')
# перезаписываем датафрейм с помощью глубокой копии, а также откидываем пустые значения в дате доставки товара клиенту
df_5['order_delivered_year_month'] = df_5['order_delivered_customer_date'].dt.strftime('%Y-%m')
# добавляем колонку в формате год-месяц, изменяя колонку даты доставки продукта
df_5.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,product_id,seller_id,shipping_limit_date,price,freight_value,weekday,order_delivered_month,days_in_month,week_in_month,order_delivered_year_month
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,...,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday,May,31.0,4.428571,2017-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,...,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday,January,31.0,4.428571,2018-01
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,...,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,Saturday,June,30.0,4.285714,2018-06
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,...,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,Tuesday,March,31.0,4.428571,2018-03
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,...,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,Sunday,August,31.0,4.428571,2018-08


In [73]:
df_5_first_order = df_5.groupby('customer_unique_id').agg({'order_delivered_customer_date': 'min'}).reset_index() \
.rename(columns={'order_delivered_customer_date': 'first_order_date'}) 
# находим самую первую дату заказа для каждого покупателя
df_5_first_order['first_order_date'] = df_5_first_order['first_order_date'].dt.strftime('%Y-%m') 
# переводим дату в удобный формат год-месяц
df_5_first_order.head()

,customer_unique_id,first_order_date
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05
2,0000f46a3911fa3c0805444483337064,2017-04
3,0000f6ccb0745a6a4b88665a16c9f078,2017-11
4,0004aac84e0df4da2b147fca70cf8255,2017-11


In [74]:
df_5_main_table = pd.merge(df_5, df_5_first_order, how='inner', on='customer_unique_id')
# присоединяем к основной таблице данные, полученные в предыдущем шаге
df_5_main_table.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,seller_id,shipping_limit_date,price,freight_value,weekday,order_delivered_month,days_in_month,week_in_month,order_delivered_year_month,first_order_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,...,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday,May,31.0,4.428571,2017-05,2017-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,...,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday,January,31.0,4.428571,2018-01,2018-01
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,...,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,Saturday,June,30.0,4.285714,2018-06,2018-06
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,...,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,Tuesday,March,31.0,4.428571,2018-03,2018-03
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,...,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,Sunday,August,31.0,4.428571,2018-08,2018-08


In [75]:
df_5_first_order_delivered_order = df_5_main_table[['customer_unique_id', 'first_order_date', 'order_delivered_year_month']] 
# отберем из общего df отдельно данные по первому заказу, а также по дате доставки
df_5_first_order_delivered_order.head()

,customer_unique_id,first_order_date,order_delivered_year_month
0,861eff4711a542e4b93843c6dd7febb0,2017-05,2017-05
1,290c77bc529b7ac935b93aa66c333dc3,2018-01,2018-01
2,060e732b5b29e8181a18229c7b0b2b5e,2018-06,2018-06
3,259dac757896d24d7702b9acbbff3f3c,2018-03,2018-03
4,345ecd01c38d18a9036ed96c73b8d066,2018-08,2018-08


In [76]:
df_5_uniq_month_costumers = df_5_first_order_delivered_order.groupby('first_order_date') \
    .agg({'customer_unique_id': 'nunique'}).reset_index() \
    .rename(columns={'customer_unique_id': 'total_uniq_costumers'}) 
# по первым датам покупки считаем кол-во уникальных покупателей по месяцам
df_5_uniq_month_costumers.head()

,first_order_date,total_uniq_costumers
0,2016-10,206
1,2016-11,59
2,2016-12,4
3,2017-01,263
4,2017-02,1325


In [77]:
df_5_data = pd.merge(df_5_first_order_delivered_order, df_5_uniq_month_costumers, how='left', on='first_order_date')
df_5_data.head() # соединяем две предыдущие таблицы

,customer_unique_id,first_order_date,order_delivered_year_month,total_uniq_costumers
0,861eff4711a542e4b93843c6dd7febb0,2017-05,2017-05,3673
1,290c77bc529b7ac935b93aa66c333dc3,2018-01,2018-01,6401
2,060e732b5b29e8181a18229c7b0b2b5e,2018-06,2018-06,6565
3,259dac757896d24d7702b9acbbff3f3c,2018-03,2018-03,6564
4,345ecd01c38d18a9036ed96c73b8d066,2018-08,2018-08,8046


In [78]:
df_5_uniq_and_total_uniq = df_5_data.groupby(['first_order_date', 'order_delivered_year_month'], as_index=False) \
    .agg({'customer_unique_id': 'nunique', 'total_uniq_costumers': 'max'}) \
    .rename(columns={'customer_unique_id': 'cohort_users'})
# с помощью группировки получаем таблицу, где отображены когорты(первый заказ) и периоды этих когорт(даты заказа пользователями)
df_5_uniq_and_total_uniq.head()

,first_order_date,order_delivered_year_month,cohort_users,total_uniq_costumers
0,2016-10,2016-10,206,206
1,2016-10,2016-11,1,206
2,2016-10,2017-04,1,206
3,2016-10,2017-08,1,206
4,2016-10,2017-10,1,206


In [79]:
df_5_cohort = df_5_uniq_and_total_uniq.query('"2016-12" < first_order_date < "2018-01"')
df_5_cohort.head()
# возьмем когорты за период 2017-2018 гг согласно условию задания

,first_order_date,order_delivered_year_month,cohort_users,total_uniq_costumers
14,2017-01,2017-01,263,263
15,2017-01,2017-02,3,263
16,2017-01,2017-03,1,263
17,2017-01,2017-05,1,263
18,2017-01,2017-06,1,263


In [80]:
df_5_cohort_copy = df_5_cohort.copy(deep=True) # делаем копию предыдущей таблицы
df_5_cohort_copy['CRR'] = (df_5_cohort.cohort_users / df_5_cohort.total_uniq_costumers * 100).round(3)
# CRR = ((Кол-во клиентов на конец периода — Новые пользователи) / Кол-во клиентов в когорте) * 100%
# Новые пользователи = Все пользователи - Кол-во пользователей из когорты (совершившие повторно покупку)
df_5_cohort_copy.head()

,first_order_date,order_delivered_year_month,cohort_users,total_uniq_costumers,CRR
14,2017-01,2017-01,263,263,100.000
15,2017-01,2017-02,3,263,1.141
16,2017-01,2017-03,1,263,0.380
17,2017-01,2017-05,1,263,0.380
18,2017-01,2017-06,1,263,0.380


In [81]:
def cohort_period(date):
    period = 0
    lst = date.split('-')
    if lst[0] == '2017':
        period = int(lst[1])
    else:
        period = int(lst[1]) + 12
    return period
# с помощью данной функции мы можем добавить в таблицу период когорт, для дальнейшего просмотра повторных покупок
df_5_cohort_copy['CohortPeriod'] = df_5_cohort_copy.order_delivered_year_month.apply(cohort_period)
# добавляем в таблицу период когорт с помощью ранее определенной функции
df_5_cohort_copy.head()

,first_order_date,order_delivered_year_month,cohort_users,total_uniq_costumers,CRR,CohortPeriod
14,2017-01,2017-01,263,263,100.000,1
15,2017-01,2017-02,3,263,1.141,2
16,2017-01,2017-03,1,263,0.380,3
17,2017-01,2017-05,1,263,0.380,5
18,2017-01,2017-06,1,263,0.380,6


In [82]:
main_table = df_5_cohort_copy.copy(deep=True).set_index(['first_order_date', 'CohortPeriod'])['CRR'].unstack(1)
# преобразовываем предыдущую таблицу и выводим итоговую
main_table

CohortPeriod,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
first_order_date,,,,,,,,,,,,,,,,,,,,,
2017-01,100.0,1.141,0.380,NaN,0.380,0.380,0.380,0.380,NaN,NaN,...,0.760,1.141,NaN,0.380,NaN,NaN,0.760,NaN,0.380,NaN
2017-02,NaN,100.000,0.679,0.226,0.302,0.302,0.151,0.151,0.226,0.075,...,0.151,0.377,0.226,0.151,0.075,0.151,0.226,NaN,0.075,NaN
2017-03,NaN,NaN,100.000,0.555,0.299,0.427,0.299,0.085,0.299,0.256,...,0.213,0.256,0.043,0.043,0.299,0.128,0.171,0.128,0.085,NaN
2017-04,NaN,NaN,NaN,100.000,0.776,0.277,0.222,0.277,0.166,0.277,...,0.388,0.222,0.277,0.111,NaN,0.111,0.111,0.166,0.166,NaN
2017-05,NaN,NaN,NaN,NaN,100.000,0.735,0.327,0.163,0.354,0.381,...,0.354,0.218,0.218,0.245,0.299,0.136,0.163,0.191,0.218,NaN
2017-06,NaN,NaN,NaN,NaN,NaN,100.000,0.704,0.576,0.448,0.352,...,0.384,0.192,0.192,0.224,0.160,0.192,0.224,0.224,0.256,NaN
2017-07,NaN,NaN,NaN,NaN,NaN,NaN,100.000,0.656,0.179,0.417,...,0.268,0.387,0.268,0.209,0.268,0.328,0.447,0.089,0.328,NaN
2017-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000,0.672,0.480,...,0.312,0.528,0.240,0.240,0.192,0.168,0.240,0.120,0.168,NaN
2017-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000,1.070,...,0.418,0.418,0.287,0.235,0.313,0.209,0.261,0.183,0.209,NaN


In [83]:
max_retention = df_5_cohort_copy # для нахождения максимального retention'a за 3-ий месяц перезаписываем таблицу
# (до основного преобразования)
max_retention.head()

,first_order_date,order_delivered_year_month,cohort_users,total_uniq_costumers,CRR,CohortPeriod
14,2017-01,2017-01,263,263,100.000,1
15,2017-01,2017-02,3,263,1.141,2
16,2017-01,2017-03,1,263,0.380,3
17,2017-01,2017-05,1,263,0.380,5
18,2017-01,2017-06,1,263,0.380,6


In [84]:
res_sp = [] # создаем пустой список для того, чтобы добавить кортеж с датой и retention
for i, row in max_retention.iterrows():
    cohort = int(row['first_order_date'].split('-')[1])
    if row['CohortPeriod'] == cohort + 3:
        res_sp.append((row['first_order_date'], row['CRR']))
max(res_sp, key=itemgetter(1)) 
# пользователи, совершившие покупку в июне 2017 года, являются когортой с самым большим retention = 0.448 на 3-ий месяц

('2017-06', 0.448)

-------------------------------------------------------------------------------------------------------------------------------

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

In [85]:
df_6 = dataset_with_customers_and_orders_items.copy(deep=True).dropna()
# создаем переменную к данному заданию, копирую основной датафрейм и избавимся от пустых значений

In [86]:
df_6.order_approved_at = pd.to_datetime(df_6.order_approved_at) 
# в связи с тем, что order_approved_at —  время подтверждения оплаты заказа, это означает, что продажа состоялась, 
# пеереводим в формат datetime

In [87]:
last_sales_day = df_6.order_approved_at.max() # определяем последнюю продажу
NOW = last_sales_day + timedelta(days=1)
# смоделируем анализ в режиме реального времени, установив дату через один день после последней покупки. Эта дата будет 
# использоваться в качестве ориентира для расчета балла NOW Recency


In [88]:
RFM = df_6.groupby('customer_unique_id') \
    .agg({'order_approved_at': lambda x: (NOW - x.max()).days, 'product_id': lambda x: len(x), 'price': lambda x: x.sum()}) \
    .rename(columns={'order_approved_at': 'R', 'product_id': 'F', 'price': 'M'})
# создаем таблицу RFM, где R - время от последней покупки пользователя до текущей даты, 
# F - суммарное количество покупок у пользователя за всё время, 
# M - сумма покупок за всё время.
RFM.head()

,R,F,M
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90
0000f46a3911fa3c0805444483337064,537,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99
0004aac84e0df4da2b147fca70cf8255,288,1,180.00


In [89]:
RFM_quantiles = RFM.quantile(q=[0.2, 0.4, 0.6, 0.8]).to_dict()
# разобьем таблицу на диапазон от 1 до 5: до 20%, 20-40%, 40-60%, 60-80% и выше 80 %
RFM_quantiles

{'R': {0.2: 92.0, 0.4: 177.0, 0.6: 268.0, 0.8: 382.0},
 'F': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'M': {0.2: 39.9, 0.4: 69.9, 0.6: 109.9, 0.8: 179.9}}

In [90]:
def to_share (x, y, z):
    if x <= z[y][0.2]:
        return 1
    elif x <= z[y][0.4]:
        return 2
    elif x <= z[y][0.6]:
        return 3
    elif x <= z[y][0.8]:
        return 4
    else:
        return 5
# определяем функцию для сегментации покупателей

In [91]:
RFM['R_segment'] = RFM['R'].apply(to_share, args=('R', RFM_quantiles, ))
RFM['F_segment'] = RFM['F'].apply(to_share, args=('F', RFM_quantiles, ))
RFM['M_segment'] = RFM['M'].apply(to_share, args=('M', RFM_quantiles, ))
# применяем функцию для каждой группы
RFM

,R,F,M,R_segment,F_segment,M_segment
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,129.90,2,1,4
0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90,2,1,1
0000f46a3911fa3c0805444483337064,537,1,69.00,5,1,2
0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99,4,1,1
0004aac84e0df4da2b147fca70cf8255,288,1,180.00,4,1,5
...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,447,2,1570.00,5,5,5
fffea47cd6d3cc0a88bd621562a9d061,262,1,64.89,3,1,2
ffff371b4d645b6ecea244b27531430a,568,1,89.90,5,1,3


In [92]:
RFM['RFM_segment'] = RFM.R_segment.map(str) + RFM.F_segment.map(str) + RFM.M_segment.map(str) # соединим все 3 сегмента в 1
RFM.reset_index().head()
# Вывод: клиенты у которых наивысший показатель RFM = 555, самые преданные и активные клиенты, которые часто совершают большие 
# покупки, показатель RFM, который стремится к RFM = 111, показывает отток клиентов, которые мало платят и редко что-либо 
# покупают. Если взять средниее значения RFM = 333, то это показывает, что клиент является лояльным, но не очень часто 
# что-либо покупает, и чек покупки не является крупным.

 

,customer_unique_id,R,F,M,R_segment,F_segment,M_segment,RFM_segment
0,0000366f3b9a7992bf8c76cfdf3221e2,112,1,129.90,2,1,4,214
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90,2,1,1,211
2,0000f46a3911fa3c0805444483337064,537,1,69.00,5,1,2,512
3,0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99,4,1,1,411
4,0004aac84e0df4da2b147fca70cf8255,288,1,180.00,4,1,5,415
